In [ ]:
from pathlib import Path
from tistools import load_path_ensembles, block_error_analysis

BASE_DIR = Path("/mnt/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations")
simulations = [
    # ("flat_w-walls", "brownian-gamma5/30k-cycles"),
    # ("flat_w-walls", "langevin-gamma5"),   
    # ("flat_w-walls", "newtonian"),                          
    # ("cosbump2-walls", "brownian"),
    ("cosbump2-walls", "langevin-gamma5"),                 
    # ("cosbump3-walls", "brownian"),  
    # ("cosbump3-walls", "langevin-gamma5"), 
    # ("cosdip2-walls", "brownian"), 
    # ("cosdip2-walls", "langevin-gamma5"), 
    # ("cosbumpmeta-walls", "brownian"), 
    # ("cosbumpmeta-walls", "brownian/fine_intf"),
    # ("cosbumpmeta-walls", "brownian/intf_shift"),
    # ("cosbumpmeta-walls", "langevin_gamma5"),
    # ("cosbumpmeta-walls", "langevin_gamma5/fine_intf"),
    # ("cosbumpmeta-walls", "langevin_gamma5/intf_shift"),
    # ("cosdipmeta-walls", "brownian"), 
    # ("cosdipmeta-walls", "langevin_gamma5"),    
]

indir_list = [BASE_DIR / sim_type / config / "REPPTIS" for sim_type, config in simulations]

for indir in indir_list:
    path_ensembles, interfaces = load_path_ensembles(indir,load=True)
    block_error_analysis(path_ensembles, interfaces, interval=10, load=False)




Setting orders for path ensemble /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/000
Loaded orders from file, NOT CHECKING FOR CONSISTENCY.
Setting orders for path ensemble /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/001
Loaded orders from file, NOT CHECKING FOR CONSISTENCY.
Setting orders for path ensemble /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/002
Loaded orders from file, NOT CHECKING FOR CONSISTENCY.
Setting orders for path ensemble /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/003
Loaded orders from file, NOT CHECKING FOR CONSISTENCY.
Setting orders for path ensemble /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/004
Loaded orders from file, NOT CHECKING FOR CONSISTENCY.
First time calculating the data file ...
Weights of the different paths:
wRMR = 0
wRML = 0
wLMR = 0
wLML = 0
Local crossing probab